In [1]:
import KT24Calculator
import requests
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
%load_ext line_profiler

In [2]:
req = requests.get('https://ktdash.app/api/killteam.php?edition=kt24')

In [3]:
operativesrows = []

for killteam in req.json():
    for fireteam in killteam['fireteams']:
        factionid = fireteam['factionid']
        if factionid == "HBR":
            continue
        killteamid = fireteam['fireteamid']
        killteamname = fireteam['fireteamname']
        for op in fireteam['operatives']:
            opid = op['opid']
            opname = op['opname']
            m = op['M'].replace('"', '')
            apl = op['APL']
            sv = op['SV'].replace('+', '')
            w = op['W']
            keywords = op['keywords'].split(' , ')
            for weapon in op['weapons']:
                for profile in weapon['profiles']:
                    wepid = weapon['wepid']
                    wepname = weapon['wepname']
                    weptype = weapon['weptype']
                    if profile['name'] != '':
                        wepname = wepname + ' (' + profile['name'] + ')'
                    A = profile['A']
                    BS = profile['BS'].replace('+', '')
                    D, Dcrit = profile['D'].split('/')
                    SR = profile['SR'].split(', ')
                    operativesrows.append([factionid, killteamid, killteamname, opid, opname, m, apl, sv, w, keywords, wepid, wepname, weptype, A, BS, D, Dcrit, SR])

In [4]:
schema = {'factionid':str, 'killteamid':str,'killteamname': str, 'opid':str, 'opname': str, 'M':pl.Int8, 'APL': pl.Int8, 'SV': pl.Int8, 'W': pl.Int8, 'keywords': list[str] ,'wepid':str, 'wepid': str, 'wepname': str, 'weptype': str, 'A': pl.Int8, 'BS': pl.Int8, 'D': pl.Int8, 'DCrit': pl.Int8, 'SR': list[str]}
operatives = pl.DataFrame(operativesrows, orient='row', schema=schema)

In [5]:
Tempestor_RelicBoltPistol = operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Tempestor", wepname='Relic Bolt Pistol').row(0, named=True)
Grenadier_MeltaBomb = operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Grenadier", wepname='Melta Bomb').row(0, named=True)
Gunfighter_Focused = operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Gunfighter", wepname='Hot-Shot Laspistol (Focused)').row(0, named=True)
Gunfighter_PointBlank =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Gunfighter", wepname='Hot-Shot Laspistol (Point Blank)').row(0, named=True)
Gunner_Melta =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Gunner", wepname='Melta Carbine').row(0, named=True)
Gunner_Plasma =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Gunner", wepname='Plasma Carbine (Standard)').row(0, named=True)
Gunner_PlasmaSuper =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Gunner", wepname='Plasma Carbine (Supercharge)').row(0, named=True)
Marksman_Stationary =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Marksman", wepname='Hot-Shot Longlas (Stationary)').row(0, named=True)
Marksman_Mobile =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Marksman", wepname='Hot-Shot Longlas (Mobile)').row(0, named=True)
Precursor_Mobile =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Precursor", wepname='Hot-Shot Laspistol').row(0, named=True)
Sentry_Flamer =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Servo-Sentry", wepname='Flamer').row(0, named=True)
Sentry_Frag =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Servo-Sentry", wepname='Grenade Launcher (Frag)').row(0, named=True)
Sentry_Krak =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Servo-Sentry", wepname='Grenade Launcher (Krak)').row(0, named=True)
Sentry_Focused =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Servo-Sentry", wepname='Hot-Shot Volley Gun (Focused)').row(0, named=True)
Sentry_sweeping =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Servo-Sentry", wepname='Hot-Shot Volley Gun (Sweeping)').row(0, named=True)
Trooper_Carbine =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Trooper", wepname='Hot-Shot Lascarbine').row(0, named=True)

Attackers = [Tempestor_RelicBoltPistol, Grenadier_MeltaBomb, Gunfighter_Focused, Gunfighter_PointBlank, Gunner_Melta, Gunner_Plasma, Gunner_PlasmaSuper, Marksman_Stationary, Marksman_Mobile, Precursor_Mobile, Sentry_Flamer, Sentry_Frag, Sentry_Krak, Sentry_Focused, Sentry_sweeping, Trooper_Carbine]

In [6]:
SM_warrior =  operatives.filter(killteamname="Angels Of Death", opname="Intercessor Warrior", wepname='Bolt Rifle').row(0, named=True)
Trooper_Carbine =  operatives.filter(killteamname="Tempestus Aquilon", opname="Aquilon Trooper", wepname='Hot-Shot Lascarbine').row(0, named=True)
DeathKorps_Trooper =  operatives.filter(killteamname="Death Korps", opname="Death Korps Trooper", wepname='Lasgun').row(0, named=True)

Defenders = [SM_warrior, Trooper_Carbine, DeathKorps_Trooper]

In [15]:
for obscured in [False, True]:
    for cover in [False, True]:
        sim = KT24Calculator.Simulation(Gunner_Melta, DeathKorps_Trooper, cover=cover, obscured=obscured)
        sim.attacker.keywords.add('Severe')
        results = sim.simulation_step(200000)
        percentage = np.array([np.sum(results == score)/len(results) for score in np.unique(results)])
        print(f"Cover: {cover}\nObscured: {obscured}\n======")
        for s, p in zip(np.unique(results), percentage):
            print(f"{s}: {p*100:.2f}")

                # results[results > sim.defender.wounds] = sim.defender.wounds

Cover: False
Obscured: False
0: 1.22
4: 1.63
7: 17.38
11: 0.32
13: 29.71
14: 3.46
18: 0.10
19: 27.03
20: 5.09
21: 0.84
25: 9.84
26: 2.72
27: 0.60
28: 0.06
Cover: True
Obscured: False
0: 1.26
7: 37.48
13: 33.55
14: 7.33
19: 14.60
20: 4.17
21: 1.53
28: 0.09
Cover: False
Obscured: True
0: 4.64
6: 16.52
12: 32.96
18: 32.82
24: 13.06
Cover: True
Obscured: True
0: 11.03
6: 29.57
12: 39.46
18: 19.94


In [ ]:
fig, axes = plt.subplots(len(Attackers), len(Defenders), figsize=(20,50))
axes = np.ravel(axes)

idx = 0

for a in Attackers:
    for d in Defenders:
        for obscured in [False, True]:
            for cover in [False, True]:
                sim = KT24Calculator.Simulation(a, d, cover=cover, obscured=obscured)
                results = sim.simulation_step(10000)
                if a['opname'] == "Aquilon Servo-Sentry":
                    results2 = sim.simulation_step(10000)
                    results = results + results2
                
                results[results > sim.defender.wounds] = sim.defender.wounds

                if not cover and not obscured:
                    label = 'Open'
                if cover and not obscured:
                    label = 'Cover'
                if not cover and obscured:
                    label = 'Obscured'
                if cover and obscured:
                    label = 'Cover & Obscured'
                axes[idx].hist(results, density=True, histtype='step', cumulative=-1, label=label)
                percentage = np.array([np.sum(results >= score)/len(results) for score in np.unique(results)])
                #axes[idx].plot(np.unique(results), percentage*100, 'o')
                if not cover and not obscured:
                    axes[idx].set_title(f"{a['opname']}({a['wepname']}) vs {d['opname']}\n Chance to: Kill:{np.sum(results >= sim.defender.wounds)/len(results)*100:.1f}%  Wound: {np.sum(results > sim.defender.wounds/2)/len(results)*100:.1f}%")
                #axes[idx].set_ylim(0,110)
                axes[idx].legend()
        idx += 1

plt.tight_layout()